# Basic Reddit Scraping
Scraping from -> r/wallstreetbets
- Daily Discussion

In [124]:
!pip install praw
!pip install get-all-tickers
!pip install tqdm
!pip install transformers
!pip install ipywidgets
!pip install IProgress
!pip install Torch
!pip install BeautifulSoup4

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [146]:
# Imports
import praw
import re
from praw.models import MoreComments
from datetime import datetime
import pandas as pd
import os
from tqdm import tqdm
from json import loads
from get_tickers import *
from threading import Thread
from requests import Session
from bs4 import BeautifulSoup
from ast import literal_eval
import torch


In [82]:
cwd = os.path.abspath('')
print(cwd)

/Users/deyuan/Documents/GitHub/sentic-ai-project/datasets


In [83]:
config = loads(open(os.path.join(cwd, 'config/config.json')).read())
reddit_config = config["Reddit"]
reddit_config

{'user_agent': 'Comment Extraction (by u/SenticFinatic)',
 'client_id': 'JVpxM3nzVundq9JhLVOU3g',
 'client_secret': 'hH-hZ2VYqhLKNDc9tt0YHYAXvs1fdw',
 'username': 'senticfinatic',
 'password': 'Sentic{h!yDIi%GKhtExCT74iz5}'}

In [84]:
reddit = praw.Reddit(
    client_id=reddit_config["client_id"],
    client_secret=reddit_config["client_secret"],
    password=reddit_config["password"],
    user_agent=reddit_config["user_agent"],
    username=reddit_config["username"]
)

In [85]:
reddit.user.me()

Redditor(name='SenticFinatic')

# Pulling data from all wsb posts excluding memes
- From start to limit

## Pulling & Packing Needed Reddit Data(From Post Comments) into DataFrame

In [86]:
sub = next(reddit.subreddit("wallstreetbets").search("Daily Discussion"))
sub

Submission(id='prqy5j')

In [8]:
comments_list = list(sub.comments).copy()
body_comments_list = []
scores_comments_list = []
id_comments_list = []
time_comments_list = []
username_comments_list = []
replies_comments_list = []
sub.comments.replace_more(limit=0)


for comment in tqdm(sub.comments):
    if isinstance(MoreComments, type(comment)):
        continue
    body_comments_list.append(comment.body)
    scores_comments_list.append(comment.score)
    try:
        id_comments_list.append(comment.author.id)
    except:
#        print(comment.body)
        id_comments_list.append("Deleted")
    try:
        username_comments_list.append(comment.author.name)
    except:
        username_comments_list.append("Deleted")
    time_comments_list.append(datetime.utcfromtimestamp(comment.created_utc)\
                              .strftime('%Y-%m-%d %H:%M:%S'))
    replies_comments_list.append(len(comment.replies))
        
    

100%|█████████████████████████████████████████| 481/481 [06:06<00:00,  1.31it/s]


In [9]:
raw_dataset = pd.DataFrame({
    "Date": time_comments_list,
    "User ID": id_comments_list,
    "Username": username_comments_list,
    "Content": body_comments_list,
    "No. of Upvotes": scores_comments_list,
    "No. of Replies": replies_comments_list,
})


## Packing Reddit Data (From Reddit Posts) to DataFrame

In [118]:
POST_LIMIT = 500

In [119]:
sub = reddit.subreddit("wallstreetbets").new(limit=POST_LIMIT)
sub

In [120]:
raw_dataset = {
    "ID":[],
    "Date": [],
    "User ID": [],
    "Username": [],
    "Title":[],
    "Content": [],
    "Upvotes":[],
    "Comments":[]
}
properties = {
    "ID":"id",
    "Date": "created_utc",
    "User ID": "author.id",
    "Username": "author.name",
    "Title":"title",
    "Content": "selftext",
    "Upvotes": "score",
    "Comments": "comments"
}

In [121]:
for post in tqdm(sub, total=POST_LIMIT):
    for col in raw_dataset:
        try:
            if("author" in properties[col]):
                ppty = post
                for subppty in properties[col].split("."):
                    ppty = getattr(ppty, subppty)
            else:
                ppty = getattr(post, properties[col])
            raw_dataset[col].append(ppty)
        except Exception as e:
            raw_dataset[col].append("[Deleted]")
    
raw_dataset = pd.DataFrame(raw_dataset)

100%|█████████████████████████████████████████| 500/500 [14:00<00:00,  1.68s/it]


In [122]:
raw_dataset

ID          Date   User ID        Username  \
0    ptqadj  1.632383e+09     qxk72    ElectroFried   
1    ptpqn3  1.632381e+09    175yyb        nosalute   
2    ptpq2s  1.632381e+09   jkktwg6       Namejamie   
3    ptpm6d  1.632380e+09  6wzaf14l     TensrWhoosh   
4    ptpm19  1.632380e+09  6ed9i8cz       Irc-freak   
..      ...           ...       ...             ...   
495  pqaxdb  1.631920e+09     3toea     space_cadet   
496  pqawik  1.631920e+09  6mayhqwz       Geoffism1   
497  pqaq4g  1.631919e+09    13a1wy       luckyrice   
498  pqamsd  1.631919e+09  a2tez47d  Johnny_Bravo_1   
499  pqafty  1.631918e+09     ngl0g       Araphoren   

                                                 Title  \
0    Reported Evergande board meeting leak happenin...   
1    Scientist discover why bears are the way they are   
2                                 CLF IS STILL RED HOT   
3          Basically sums up most of retarded WSB apes   
4                              $sofi gains. 2 hr hold.   
..                                                 ...   
495                    Shorting China with $37k - YOLO   
496  CLF was on sale today! $22,887 In Jan 2022 $25...   
497                       SoFi Basically All In Hoorah   
498                                                  👀   
499                   SDC - Dehedging the (tiny) float   

                                               Content  Upvotes  \
0                                                            25   
1                                                            38   
2    CLF or Cleveland-Cliffs is the largest flat ro...        6   
3                                                            11   
4                                                            15   
..                                                 ...      ...   
495                                                           2   
496                                                          38   
497                                                         106   
498                                                          92   
499  Are we done?\n\nNo, of course not.\n\nDidn't h...     2866   

                                              Comments  
0    (hdxurxw, hdxvbl2, hdxuxl3, hdxv0db, hdxv1gz, ...  
1                 (hdxrqag, hdxs8zq, hdxu3b3, hdxvwm7)  
2                                   (hdxrn7z, hdxrpq6)  
3        (hdxr30b, hdxr9s9, hdxrj4w, hdxskwl, hdxvzkq)  
4                                            (hdxr1hp)  
..                                                 ...  
495                        (hd9m1zy, hdjynkr, hdl65ug)  
496  (hd9lw26, hd9pt4n, hd9s35v, hd9mai4, hd9paqm, ...  
497  (hd9kkmn, hd9lq7y, hda1k4z, hd9kvzy, hda4onz, ...  
498  (hd9jv0j, hd9kjom, hd9k92l, hd9o9ly, hd9kd87, ...  
499  (hd9iivc, hd9vb38, hd9l4yy, hd9ni49, hd9mhcp, ...  

[500 rows x 8 columns]

In [123]:
raw_dataset.to_csv("raw_reddit_post_dataset.csv") 

# Data Cleaning (Using context parsing)

Instead of using this senticnet API for solely analysis, we can just context parsing to remove unnecessary data for ticker(stock symbol) filter

In [2]:
raw_dataset = pd.read_csv("raw_reddit_dataset.csv") # Loading previously scraped dataset
raw_dataset

Unnamed: 0                 Date   User ID              Username  \
0             0  2021-09-20 20:16:31     4nc92                   zjz   
1             1  2021-09-20 11:02:25    1293gu             picciona4   
2             2  2021-09-20 10:16:54    15tqn5             nutfugget   
3             3  2021-09-20 11:44:01     kav2g        EatMaTesticles   
4             4  2021-09-20 11:12:21  2eptxwqx             dela_sole   
..          ...                  ...       ...                   ...   
476         476  2021-09-20 12:05:38     bz90j            Ruckefella   
477         477  2021-09-20 12:06:36     4fm5x               Shmoogy   
478         478  2021-09-20 12:08:21  9xewy948      Fragrant-War-344   
479         479  2021-09-20 12:10:43   i8sxc86  MakIkEenDonerMetKalf   
480         480  2021-09-20 12:16:31     6k71v                 j4k3b   

                                               Content  No. of Upvotes  \
0    New Thread REEEEEEEEEEEEEEEEEEEEEEEEEEEE\n\nht...               1   
1    Man, good thing those FED officials sold every...             226   
2    The Fed presidents disclosed they sold their s...             219   
3    This is the fifth once in a lifetime opportuni...             203   
4           Stupid evergrande blocking the river 🤦🏼‍♂️             152   
..                                                 ...             ...   
476  I got a great nights sleep last night lads. To...              11   
477  The fact that everybody is so bearish ... i th...              11   
478  Dam happy I am holding 20% of my portfolio in ...              12   
479                          The fed sold the top lmao              10   
480  Oh look another incident where I get fucked in...              10   

     No. of Replies  
0                 0  
1                 0  
2                 0  
3                 1  
4                 1  
..              ...  
476               0  
477               0  
478               0  
479               0  
480               0  

[481 rows x 7 columns]

## Splitting Data

In [8]:
"""
In order to increase cleaning efficeny, utilise multithreading to
perform multiple requests at the same time to SenticNet API
"""
THREADS = 5
clean_ds = [None] * THREADS
def split_dataset(data, n):
    length = len(data)
    step = int(length / n)
    for x in range(step, length, step):
        if length - x < step:
            yield data.iloc[x-step:]
        yield data.iloc[x-step:x]
     
#    for x in range(len(data)):
splitted_ds = list(split_dataset(raw_dataset, THREADS))
splitted_ds

[    Unnamed: 0                 Date   User ID        Username  \
 0            0  2021-09-20 20:16:31     4nc92             zjz   
 1            1  2021-09-20 11:02:25    1293gu       picciona4   
 2            2  2021-09-20 10:16:54    15tqn5       nutfugget   
 3            3  2021-09-20 11:44:01     kav2g  EatMaTesticles   
 4            4  2021-09-20 11:12:21  2eptxwqx       dela_sole   
 ..         ...                  ...       ...             ...   
 91          91  2021-09-20 17:34:36     xmdqk            fufm   
 92          92  2021-09-20 09:55:32  3pjeo05f  PorkchopSquats   
 93          93  2021-09-20 11:36:19     dhyzi           talm0   
 94          94  2021-09-20 11:44:11     vling         Tsad311   
 95          95  2021-09-20 12:32:01     wjmku      WhatsMyMDA   
 
                                               Content  No. of Upvotes  \
 0   New Thread REEEEEEEEEEEEEEEEEEEEEEEEEEEE\n\nht...               1   
 1   Man, good thing those FED officials sold every...    

In [9]:
from transformers import pipeline

summarizer = pipeline("summarization")


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


In [10]:
def Summarise(dataset : pd.DataFrame, thread_num : int)-> pd.DataFrame:
    """
    Input:
    Dataset with Content Column
    Return:
    Dataset with modified Content Column
    """
    cleaned_ds = []
    for row in dataset.iloc():
        row["Content"] = summarizer(row["Content"])[0]["summary_text"]
        cleaned_ds.append(row)
    clean_ds[thread_num] = cleaned_ds

In [41]:
threads = []
for x in range(THREADS):
    threads.append(Thread(target=Summarise,args=(splitted_ds[x],x,)))
print(threads)
for x in range(THREADS):
    threads[x].start()
for x in range(THREADS):
    threads[x].join()
clean_ds = pd.DataFrame(clean_ds)
print("Completed")
clean_ds


Your max_length is set to 142, but you input_length is only 57. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 56, but you input_length is only 13. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your min_length is set to 56, but you input_length is only 21. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your min_length is set to 56, but you input_length is only 7. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your min_length is set to 56, but you input_length is only 12. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 13. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 21. You might consider decr

[<Thread(Thread-19, initial)>, <Thread(Thread-20, initial)>, <Thread(Thread-21, initial)>, <Thread(Thread-22, initial)>, <Thread(Thread-23, initial)>]


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()
Your min_length is set to 56, but you input_length is only 19. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 19. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 56, but you input_length is only 27. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 27. You might consider decreasing max_length manually, e.g. summarizer('...

Your min_length is set to 56, but you input_length is only 17. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 17. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 56, but you input_length is only 20. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 20. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 56, but you input_length is only 24. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 24. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 56, but you input_length is only 11. You might consider dec

Your max_length is set to 142, but you input_length is only 29. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 56, but you input_length is only 15. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 15. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 56, but you input_length is only 23. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 23. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 30. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 75. You might consider d

...

Your max_length is set to 142, but you input_length is only 19. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 56, but you input_length is only 12. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 12. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 56, but you input_length is only 15. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 15. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 56, but you input_length is only 6. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 6. You might consider decr

Your max_length is set to 142, but you input_length is only 24. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 56, but you input_length is only 22. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 22. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 56, but you input_length is only 6. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 6. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 56, but you input_length is only 13. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 13. You might consider decr

Your max_length is set to 142, but you input_length is only 49. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 56, but you input_length is only 24. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 24. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 40. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 45. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 56, but you input_length is only 21. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 21. You might consider d

Your max_length is set to 142, but you input_length is only 12. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 56, but you input_length is only 6. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 6. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 37. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but you input_length is only 49. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your min_length is set to 56, but you input_length is only 23. You might consider decreasing min_length manually, e.g. summarizer('...', min_length=10)
Your max_length is set to 142, but you input_length is only 23. You might consider dec

Completed


In [139]:
sentic_config = config["SenticNet"]
CP_API = sentic_config["Concept_Parsing"]
CP_API

'z6S2XFFp4rZ1F'

In [164]:
def SenticNet(dataset : pd.DataFrame, API_KEY : str, col_name : str , LANGUAGE="en")->pd.DataFrame:
    """
    Require: 
    1) Dataset with ["Content"] column
    2) API Key for SenticNet API
    3) Column Name for data obtained from SenticNet API
    Returns:
    Dataframe with ["Cleaned Content" : List] Column
    """
    s = Session()
    url = f"https://sentic.net/api/{LANGUAGE}/{API_KEY}.py"
    new_ds = []
    for row in dataset.iloc():
        params = {
            "text" : row["Content"].replace(" ", "%20")
        }
        res = s.get(url, params=params)
        row[col_name] = res.text
        new_ds.append(row)
    return pd.DataFrame(new_ds)


First it was Evergreen

Now it's Evergrande

What the Everfuck is next?
No concepts detected



In [160]:
threads = []
SenticNet(raw_dataset, CP_API)
    

This is the fifth once in a lifetime opportunity for my generation
['lifetime']



# Harvesting off ticker counts without sentiment analysis
- In sentiment analysis, we would give a score for the top 5 root comments of the comment/post rrelated to the position
- This ticker counter does not include that

In [171]:
def get_stock_tickers():
    s = Session()
    url = "https://api.stockanalysis.com/wp-json/sa/search?q=index"
    stocks = literal_eval(s.get(url).text)
    tickers = {
        "ticker":[],
        "Name":[],
    }
    for stock in tqdm(stocks):
        tickers["ticker"].append(stock["s"])
        tickers["Name"].append(stock["n"])
    return tickers
tickers = pd.DataFrame(get_stock_tickers())
tickers
    

100%|██████████████████████████████████| 9437/9437 [00:00<00:00, 4024160.92it/s]


ticker                                               Name
0         A                               Agilent Technologies
1        AA                                              Alcoa
2       AAC                                   Ares Acquisition
3      AACE              Alexandria Agtech\/Climate Innovation
4      AACG                              ATA Creativity Global
...     ...                                                ...
9432   ZHDG                             ZEGA Buy and Hedge ETF
9433    ZIG                                     Acquirers Fund
9434   ZJPN                           SPDR Solactive Japan ETF
9435   ZROZ  PIMCO 25 Plus Year Zero Coupon U.S. Treasury I...
9436    ZSL                        ProShares UltraShort Silver

[9437 rows x 2 columns]

In [161]:
tickers.to_csv("tickers.csv")

In [172]:
filtered_dataset = {
    "Comment/Post": [],
    "Corresponding ticker": []
}
patterns = {ticker:re.compile(f"\W{ticker}\W") for ticker in tickers["ticker"].iloc()}
patterns

ds = raw_dataset
col = "Title" # Col with content to find/extract tickers
for x in tqdm(range(len(ds[col]))):
    tkers = []
    for ticker in patterns:
        results = re.search(patterns[ticker], ds[col][x])
        if results:
            tkers.append(ticker)
    if len(tkers) != 0:
        filtered_dataset["Comment/Post"].append(ds.iloc[x])
        filtered_dataset["Corresponding ticker"].append(tkers)


{'A': re.compile(r'\WA\W', re.UNICODE),
 'AA': re.compile(r'\WAA\W', re.UNICODE),
 'AAC': re.compile(r'\WAAC\W', re.UNICODE),
 'AACE': re.compile(r'\WAACE\W', re.UNICODE),
 'AACG': re.compile(r'\WAACG\W', re.UNICODE),
 'AACI': re.compile(r'\WAACI\W', re.UNICODE),
 'AACO': re.compile(r'\WAACO\W', re.UNICODE),
 'AACP': re.compile(r'\WAACP\W', re.UNICODE),
 'AADI': re.compile(r'\WAADI\W', re.UNICODE),
 'AAIC': re.compile(r'\WAAIC\W', re.UNICODE),
 'AAL': re.compile(r'\WAAL\W', re.UNICODE),
 'AAMC': re.compile(r'\WAAMC\W', re.UNICODE),
 'AAME': re.compile(r'\WAAME\W', re.UNICODE),
 'AAN': re.compile(r'\WAAN\W', re.UNICODE),
 'AAOI': re.compile(r'\WAAOI\W', re.UNICODE),
 'AAON': re.compile(r'\WAAON\W', re.UNICODE),
 'AAP': re.compile(r'\WAAP\W', re.UNICODE),
 'AAPL': re.compile(r'\WAAPL\W', re.UNICODE),
 'AAQC': re.compile(r'\WAAQC\W', re.UNICODE),
 'AAT': re.compile(r'\WAAT\W', re.UNICODE),
 'AATC': re.compile(r'\WAATC\W', re.UNICODE),
 'AAU': re.compile(r'\WAAU\W', re.UNICODE),
 'AAWW': r

In [175]:
fd = pd.DataFrame(filtered_dataset["Comment/Post"])
fd["Stock"] = filtered_dataset["Corresponding ticker"]

In [183]:
a = []
for x in range(len(filtered_dataset["Corresponding ticker"])):
    for y in filtered_dataset["Corresponding ticker"][x]:
        if "SPY"  == y:
            a.append(fd.iloc[x])
            break
a = pd.DataFrame(a)
a

ID          Date   User ID              Username  \
19   ptmnsj  1.632368e+09  a15t43vb            wsbmuskrat   
20   ptmh5d  1.632367e+09  1onht8u1                atac03   
36   ptjheu  1.632357e+09    13bph0           cashbackpal   
108  pt8av8  1.632320e+09  5x9cxmk5          globohydrate   
186  psr7ds  1.632256e+09  9xkvswtl              rebelo55   
204  pso9zs  1.632247e+09  4dwj5987            OTMorDeath   
237  psjp2c  1.632234e+09  a3gcdrz9           BFLO-Retail   
269  psb5h0  1.632197e+09   l2i7wdb    Mcdaddybigballs602   
285  ps5yav  1.632179e+09  5vou12u8             ROCKETWAE   
290  ps56w7  1.632176e+09    w5brib                mpoozd   
328  ps0mct  1.632162e+09  32puvype         BearIncarnate   
353  prwe7g  1.632149e+09  4lfit3b6  Remarkable_Health722   
385  pro6ya  1.632115e+09  58lkkbu7            StarSwitch   
455  pqpq27  1.631983e+09     g0xvh             Berisha11   
470  pqk0jp  1.631961e+09  51qfxpzq          nazarath6969   

                                                 Title  \
19      A friend, Never bet against America. $SPY PUTS   
20   SOFI, ATVI, EA, SPY, WFC YOLO. VERY dIvERsiFIE...   
36   Got rekt by YANG today (-5k), day traded my wa...   
108                                  26k SPY 0DTE yolo   
186                                         $SPY today   
204  Update from mini SPY YOLO (if anyone cares) so...   
237  How to short SPY with Synthetic Short Shares C...   
269           Sportsbook doesn’t work like $SPY calls.   
285                    People with SPY calls right now   
290  This is why SPY skyrocketed in the last 30 min...   
328  You may remember me as the guy who lost 28.6k ...   
353  $SPY pressure continuation ofter breaking bear...   
385   Options Hedging for 20 SEP 21: SDC IRNT GME SPY+   
455  How $SPY puts made me find the woman of my dre...   
470  6k -> 73k -> 1k all in a month with all my los...   

                                               Content  Upvotes  \
19                                                         1248   
20                                                           15   
36                                                           36   
108                                                          32   
186                                                       13569   
204                                                          27   
237  What are SPY synthetic short shares?? (and why...       12   
269                                                          31   
285                                                         527   
290                                                         399   
328                                                       18231   
353  We did well to go aggressively bearish last we...       17   
385  **TLDR: Stop gambling you allowance money with...      287   
455  I still can't believe this guys, but I've fina...      250   
470                                                         404   

                                              Comments  \
19   (hdx8uvm, hdx99f5, hdx9omh, hdx9glj, hdxacao, ...   
20                (hdx7lvf, hdx7twv, hdx7uzr, hdxa2rk)   
36   (hdwlq56, hdwlx0a, hdwohix, hdwx2c9, hdwm48b, ...   
108  (hduhpug, hdui6v0, hduinu7, hduup5u, hduipy0, ...   
186  (hdrfgh5, hdrjppo, hdrgbmh, hdrn9sd, hdroiye, ...   
204  (hdquv8m, hdqwb5u, hdr0q5l, hdr5b8q, hdrb4xs, ...   
237  (hdpznnb, hdqack7, hdq1rch, hdq5ulf, hdqzbvl, ...   
269  (hdoid9k, hdoiggj, hdokkbw, hdoizdm, hdokm96, ...   
285  (hdng2wx, hdnhm3v, hdnj4eo, hdnl72n, hdnzax8, ...   
290  (hdnada9, hdnac5j, hdnlakp, hdnam2o, hdni1mo, ...   
328  (hdmd15f, hdmd4ul, hdmh7j8, hdmrg2b, hdmgcoj, ...   
353  (hdlhkvq, hdlj70w, hdljc6q, hdlikmm, hdli46f, ...   
385  (hdk2saq, hdk3j3c, hdk5f2j, hdk3tkc, hdk5j19, ...   
455  (hdck9bs, hdckvo7, hdct358, hdcqbyy, hdcsokf, ...   
470  (hdbeid9, hdbfq4y, hdbl3oh, hdbgxqo, hdbvkt4, ...   

                                Stock  
19                              [SPY]  
20   [ATVI, EA, VERY,